In [13]:
from dotenv import load_dotenv
load_dotenv() 

True

In [14]:
import tiktoken
from PyPDF2 import PdfReader
import os
from openai import OpenAI


In [5]:
pdf_path = "example_docs/sample_med2.pdf"

In [6]:
with open(pdf_path, 'rb') as file:
    reader = PdfReader(file)
    num_pages = len(reader.pages)
    text = ""
    inside_section = False
    pages = []
    for i in range(num_pages):
        page = reader.pages[i]
        page_text = page.extract_text()
        pages.append(page_text)

In [7]:
sumLength = 0
for page in pages: 
    sumLength += len(page)

summed_doc = "\n".join(pages)

In [8]:
enc = tiktoken.encoding_for_model("gpt-4o")

In [9]:
res = enc.encode(summed_doc)

In [10]:
len(res)

11863

In [33]:
client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": f"summarize this paper for me please: ```{summed_doc[:1000]}```. response must format in json format with key as 'summary'",
        }
    ],
    model="gpt-4o",
    response_format={"type": "json_object"},
)

In [34]:
print(chat_completion.choices[0].message.content)

{
  "summary": "This paper investigates whether the timing of telmisartan administration (morning versus evening) impacts its efficacy in treating essential hypertension. Prior research has shown that valsartan, another angiotensin receptor blocker (ARB), is more effective when taken at bedtime rather than on waking, leading to better blood pressure control during sleep without losing overall 24-hour efficacy. The study aims to determine if this effect is class-specific to ARBs or unique to valsartan. Given that telmisartan has a longer terminal half-life compared to other ARBs, the study evaluates its time-dependent antihypertensive effects in 215 hypertensive patients (114 men and 101 women, average age 46.4 years), who were randomly assigned to take 80 mg of telmisartan daily either on awakening or at bedtime."
}


In [35]:
import json 

In [36]:
json.loads(chat_completion.choices[0].message.content)

{'summary': 'This paper investigates whether the timing of telmisartan administration (morning versus evening) impacts its efficacy in treating essential hypertension. Prior research has shown that valsartan, another angiotensin receptor blocker (ARB), is more effective when taken at bedtime rather than on waking, leading to better blood pressure control during sleep without losing overall 24-hour efficacy. The study aims to determine if this effect is class-specific to ARBs or unique to valsartan. Given that telmisartan has a longer terminal half-life compared to other ARBs, the study evaluates its time-dependent antihypertensive effects in 215 hypertensive patients (114 men and 101 women, average age 46.4 years), who were randomly assigned to take 80 mg of telmisartan daily either on awakening or at bedtime.'}

In [31]:
chat_completion.usage.total_tokens

12383